# BGP Label Validation & Anomaly Discovery

## Purpose
This notebook helps you **validate and discover labels** in your BGP/RIPE data when you don't have ground truth.

## The Problem
You have RIPE data that you assume is "normal", but you don't actually know:
- Is it truly normal traffic?
- Does it contain hidden anomalies or attacks?
- How can you trust your labels?

## The Solution
We use **multiple unsupervised methods** and **consensus voting** to:
1. Discover natural clusters in your data
2. Find anomalies using 5 different algorithms
3. Generate confidence-based labels
4. Visualize the results

---

## 1. Setup & Configuration

In [ ]:
# Core imports
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Machine Learning
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor, NearestNeighbors
from sklearn.cluster import DBSCAN, KMeans
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, calinski_harabasz_score
from sklearn.covariance import EllipticEnvelope
from scipy import stats

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Set plot style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

print("All imports successful!")

In [ ]:
# ============================================
# CONFIGURATION - MODIFY THESE VALUES
# ============================================

# Path to your features CSV file
INPUT_FILE = "/path/to/your/features.csv"  # <-- CHANGE THIS

# Output directory (leave None to save in same directory as input)
OUTPUT_DIR = None

# Estimated anomaly rate (0.1 = 10%)
# If you have no idea, start with 0.1
CONTAMINATION_ESTIMATE = 0.10

# Label column name (if you have existing labels)
EXISTING_LABEL_COL = 'label'  # Set to None if no existing labels

# Time column name (for temporal analysis)
TIME_COL = 'window_start'  # Set to None if no time column

# Random seed for reproducibility
RANDOM_STATE = 42

print(f"Configuration set:")
print(f"  Input file: {INPUT_FILE}")
print(f"  Contamination estimate: {CONTAMINATION_ESTIMATE*100:.0f}%")

## 2. Load and Explore Data

In [ ]:
# Load the data
df = pd.read_csv(INPUT_FILE)

print(f"Loaded {len(df)} samples")
print(f"Columns: {len(df.columns)}")
print(f"\nColumn names:")
for i, col in enumerate(df.columns):
    print(f"  {i+1}. {col}")

In [ ]:
# Basic statistics
print("Data Overview:")
print("="*60)
df.info()
print("\n")
df.describe()

In [ ]:
# Check existing labels if present
if EXISTING_LABEL_COL and EXISTING_LABEL_COL in df.columns:
    print(f"Existing label distribution ({EXISTING_LABEL_COL}):")
    print("="*40)
    label_counts = df[EXISTING_LABEL_COL].value_counts()
    for label, count in label_counts.items():
        pct = count / len(df) * 100
        print(f"  {label}: {count} ({pct:.1f}%)")
    
    # Visualize
    plt.figure(figsize=(10, 5))
    label_counts.plot(kind='bar', color=sns.color_palette("husl", len(label_counts)))
    plt.title('Existing Label Distribution')
    plt.xlabel('Label')
    plt.ylabel('Count')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
else:
    print("No existing labels found - this is fine!")
    print("We will discover labels from the data.")

## 3. Prepare Features

In [ ]:
# Identify feature columns (exclude metadata)
META_COLS = {
    'incident', 'window_start', 'window_end', 'timestamp', 'time',
    'label', 'label_rule', 'label_refined', 'label_discovered',
    'cluster', 'anomaly_score', 'source', 'collector'
}

# Get numeric columns that are not metadata
candidate_cols = [c for c in df.columns if c.lower() not in {m.lower() for m in META_COLS}]
feature_cols = df[candidate_cols].select_dtypes(include=[np.number]).columns.tolist()

print(f"Identified {len(feature_cols)} feature columns:")
for i, col in enumerate(feature_cols):
    print(f"  {i+1}. {col}")

In [ ]:
# Prepare feature matrix
X = df[feature_cols].values

# Handle missing values
valid_mask = ~np.isnan(X).any(axis=1)
X_valid = X[valid_mask]

print(f"Valid samples: {len(X_valid)} / {len(X)}")
print(f"Excluded due to NaN: {(~valid_mask).sum()}")

# Scale features (RobustScaler is robust to outliers)
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X_valid)

print(f"\nFeature matrix shape: {X_scaled.shape}")

## 4. Hyperparameter Tuning & Explanation

Before running anomaly detection, let's understand and tune the key parameters.

### Parameter Reference Table

| Parameter | Default | Range | How to Choose |
|-----------|---------|-------|---------------|
| `n_estimators` (IF) | 200 | 100-500 | More = stable but slower. 200 is usually enough |
| `n_neighbors` (LOF) | 20 | 5-50 | √n_samples or use tuning below |
| `IQR multiplier` | 1.5 | 1.5-3.0 | 1.5=standard, 3.0=very conservative |
| `PCA components` | 10 | 5-20 | Preserve ~95% variance |
| `DBSCAN eps` | auto | varies | Use k-distance graph |
| `min_samples` (DBSCAN) | 5 | 3-10 | Higher = fewer clusters |

In [ ]:
# ============================================
# HYPERPARAMETER CONFIGURATION
# ============================================
# These are the key parameters you can tune

# 1. Isolation Forest
N_ESTIMATORS = 200  # Number of trees (100-500, more = stable but slower)

# 2. Local Outlier Factor  
# Rule of thumb: sqrt(n_samples) or between 10-50
N_NEIGHBORS_LOF = max(10, min(50, int(np.sqrt(len(X_valid)))))

# 3. Statistical Outliers
IQR_MULTIPLIER = 1.5  # Standard=1.5, Conservative=2.0, Very Conservative=3.0
Z_SCORE_THRESHOLD = 3  # Standard=3, Conservative=2.5

# 4. PCA Components (for DBSCAN)
# Will be set based on variance explained

# 5. DBSCAN
MIN_SAMPLES_DBSCAN = 5  # Minimum points to form a cluster

print("Hyperparameters configured:")
print(f"  Isolation Forest n_estimators: {N_ESTIMATORS}")
print(f"  LOF n_neighbors: {N_NEIGHBORS_LOF} (based on sqrt({len(X_valid)}))")
print(f"  IQR multiplier: {IQR_MULTIPLIER}")
print(f"  Z-score threshold: {Z_SCORE_THRESHOLD}")
print(f"  DBSCAN min_samples: {MIN_SAMPLES_DBSCAN}")

In [ ]:
# Tune LOF n_neighbors using stability analysis
print("Tuning LOF n_neighbors...")

neighbor_range = [5, 10, 15, 20, 30, 40, 50]
neighbor_range = [n for n in neighbor_range if n < len(X_valid) // 2]  # Must be less than n_samples/2

lof_stability = []
for n in neighbor_range:
    lof_temp = LocalOutlierFactor(n_neighbors=n, contamination=CONTAMINATION_ESTIMATE)
    preds = lof_temp.fit_predict(X_scaled)
    anomaly_rate = (preds == -1).mean()
    lof_stability.append((n, anomaly_rate))
    print(f"  n_neighbors={n}: {anomaly_rate*100:.1f}% anomalies")

# Plot
plt.figure(figsize=(10, 4))
ns, rates = zip(*lof_stability)
plt.plot(ns, [r*100 for r in rates], 'bo-', markersize=8)
plt.axhline(y=CONTAMINATION_ESTIMATE*100, color='red', linestyle='--', 
            label=f'Expected: {CONTAMINATION_ESTIMATE*100:.0f}%')
plt.xlabel('n_neighbors')
plt.ylabel('Anomaly Rate (%)')
plt.title('LOF Sensitivity to n_neighbors\n(Choose value where curve stabilizes)')
plt.legend()
plt.grid(True)
plt.show()

# Find most stable value (closest to expected contamination)
best_n = min(lof_stability, key=lambda x: abs(x[1] - CONTAMINATION_ESTIMATE))[0]
print(f"\nRecommended n_neighbors: {best_n}")

In [ ]:
# Determine optimal PCA components (preserve 95% variance)
print("Analyzing PCA variance...")

pca_full = PCA()
pca_full.fit(X_scaled)

cumulative_variance = np.cumsum(pca_full.explained_variance_ratio_)

# Find number of components for 95% variance
n_components_95 = np.argmax(cumulative_variance >= 0.95) + 1
n_components_90 = np.argmax(cumulative_variance >= 0.90) + 1

print(f"  Components for 90% variance: {n_components_90}")
print(f"  Components for 95% variance: {n_components_95}")

# Plot
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# Individual variance
axes[0].bar(range(1, len(pca_full.explained_variance_ratio_)+1), 
            pca_full.explained_variance_ratio_*100)
axes[0].set_xlabel('Principal Component')
axes[0].set_ylabel('Variance Explained (%)')
axes[0].set_title('Variance per Component')

# Cumulative variance
axes[1].plot(range(1, len(cumulative_variance)+1), cumulative_variance*100, 'b-', linewidth=2)
axes[1].axhline(y=95, color='red', linestyle='--', label='95% threshold')
axes[1].axhline(y=90, color='orange', linestyle='--', label='90% threshold')
axes[1].axvline(x=n_components_95, color='red', linestyle=':', alpha=0.5)
axes[1].axvline(x=n_components_90, color='orange', linestyle=':', alpha=0.5)
axes[1].set_xlabel('Number of Components')
axes[1].set_ylabel('Cumulative Variance (%)')
axes[1].set_title('Cumulative Variance Explained')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

# Set optimal PCA components
N_PCA_COMPONENTS = n_components_95
print(f"\nUsing {N_PCA_COMPONENTS} PCA components (95% variance)")

In [ ]:
# HDBSCAN doesn't need eps tuning!
# This cell is now optional - just shows the k-distance graph for reference

print("=" * 60)
print("NOTE: We're using HDBSCAN instead of DBSCAN")
print("HDBSCAN doesn't require eps tuning - it's automatic!")
print("=" * 60)
print("\nSkipping eps tuning (not needed for HDBSCAN)")
print("You can delete this cell if you want.")

# Set a placeholder for compatibility
DBSCAN_EPS = None  # Not used with HDBSCAN

## 5. Anomaly Detection Methods (with tuned parameters)

Now we run the 5 methods using the parameters we tuned above.

In [ ]:
# Method 1: Isolation Forest (using tuned parameters)
print("[1/5] Running Isolation Forest...")

iso_forest = IsolationForest(
    n_estimators=N_ESTIMATORS,  # Using tuned parameter
    contamination=CONTAMINATION_ESTIMATE,
    random_state=RANDOM_STATE,
    n_jobs=-1
)

iso_predictions = iso_forest.fit_predict(X_scaled)
iso_scores = iso_forest.decision_function(X_scaled)

iso_anomalies = (iso_predictions == -1)
print(f"  Parameters: n_estimators={N_ESTIMATORS}")
print(f"  Found {iso_anomalies.sum()} anomalies ({iso_anomalies.mean()*100:.1f}%)")

In [ ]:
# Method 2: Local Outlier Factor (using tuned parameters)
print("[2/5] Running Local Outlier Factor...")

lof = LocalOutlierFactor(
    n_neighbors=N_NEIGHBORS_LOF,  # Using tuned parameter
    contamination=CONTAMINATION_ESTIMATE,
    novelty=False
)

lof_predictions = lof.fit_predict(X_scaled)
lof_scores = lof.negative_outlier_factor_

lof_anomalies = (lof_predictions == -1)
print(f"  Parameters: n_neighbors={N_NEIGHBORS_LOF}")
print(f"  Found {lof_anomalies.sum()} anomalies ({lof_anomalies.mean()*100:.1f}%)")

In [ ]:
# Method 3: Statistical Analysis (using tuned parameters)
print("[3/5] Running Statistical Analysis...")

def statistical_outlier_scores(X, z_threshold=3, iqr_mult=1.5):
    """Calculate outlier scores using Z-score and IQR."""
    n_samples, n_features = X.shape
    outlier_scores = np.zeros(n_samples)
    
    for i in range(n_features):
        col = X[:, i]
        
        # Z-score method (tunable threshold)
        z_scores = np.abs(stats.zscore(col, nan_policy='omit'))
        z_outliers = z_scores > z_threshold
        
        # IQR method (tunable multiplier)
        Q1, Q3 = np.percentile(col, [25, 75])
        IQR = Q3 - Q1
        iqr_outliers = (col < Q1 - iqr_mult * IQR) | (col > Q3 + iqr_mult * IQR)
        
        outlier_scores += z_outliers.astype(float) + iqr_outliers.astype(float)
    
    return outlier_scores / (2 * n_features)

# Use tuned parameters
stat_scores = statistical_outlier_scores(X_valid, 
                                          z_threshold=Z_SCORE_THRESHOLD,
                                          iqr_mult=IQR_MULTIPLIER)
stat_threshold = np.percentile(stat_scores, 100 * (1 - CONTAMINATION_ESTIMATE))
stat_anomalies = stat_scores > stat_threshold

print(f"  Parameters: Z-score threshold={Z_SCORE_THRESHOLD}, IQR multiplier={IQR_MULTIPLIER}")
print(f"  Found {stat_anomalies.sum()} anomalies ({stat_anomalies.mean()*100:.1f}%)")

In [ ]:
# Method 4: Elliptic Envelope (Robust Covariance)
print("[4/5] Running Elliptic Envelope...")

try:
    elliptic = EllipticEnvelope(
        contamination=CONTAMINATION_ESTIMATE,
        random_state=RANDOM_STATE
    )
    elliptic_predictions = elliptic.fit_predict(X_scaled)
    elliptic_scores = elliptic.decision_function(X_scaled)
    elliptic_anomalies = (elliptic_predictions == -1)
    print(f"  Found {elliptic_anomalies.sum()} anomalies ({elliptic_anomalies.mean()*100:.1f}%)")
    elliptic_success = True
except Exception as e:
    print(f"  Skipped due to error: {e}")
    elliptic_anomalies = np.zeros(len(X_valid), dtype=bool)
    elliptic_scores = np.zeros(len(X_valid))
    elliptic_success = False

In [ ]:
# Method 5: HDBSCAN (better than DBSCAN - no eps tuning needed!)
print("[5/5] Running HDBSCAN Clustering...")

# Install hdbscan if not available
try:
    import hdbscan
except ImportError:
    print("Installing hdbscan...")
    import subprocess
    subprocess.check_call(['pip', 'install', 'hdbscan', '-q'])
    import hdbscan

import gc

# Use PCA with tuned components
pca = PCA(n_components=N_PCA_COMPONENTS)
X_pca = pca.fit_transform(X_scaled)
print(f"  PCA variance explained: {sum(pca.explained_variance_ratio_)*100:.1f}%")

# HDBSCAN parameters
MIN_CLUSTER_SIZE = max(50, int(len(X_pca) * 0.005))  # At least 0.5% of data or 50
MIN_SAMPLES = MIN_SAMPLES_DBSCAN  # Reuse the parameter we set

print(f"  Running HDBSCAN on {len(X_pca)} samples...")
print(f"  Parameters: min_cluster_size={MIN_CLUSTER_SIZE}, min_samples={MIN_SAMPLES}")

# HDBSCAN - much better than DBSCAN!
clusterer = hdbscan.HDBSCAN(
    min_cluster_size=MIN_CLUSTER_SIZE,
    min_samples=MIN_SAMPLES,
    cluster_selection_method='eom',  # Excess of Mass (better for anomaly detection)
    prediction_data=False,  # Save memory
    core_dist_n_jobs=-1  # Use all cores
)

hdbscan_labels = clusterer.fit_predict(X_pca)

# Get outlier scores (probability of being an outlier)
# Higher score = more likely to be outlier
outlier_scores = clusterer.outlier_scores_

# Clean up
gc.collect()

# In HDBSCAN, -1 means noise/outlier
hdbscan_anomalies = (hdbscan_labels == -1)
n_clusters = len(set(hdbscan_labels)) - (1 if -1 in hdbscan_labels else 0)

print(f"  Found {n_clusters} clusters")
print(f"  Found {hdbscan_anomalies.sum()} outliers ({hdbscan_anomalies.mean()*100:.1f}%)")

# Store for later use (HDBSCAN provides better outlier scores)
dbscan_anomalies = hdbscan_anomalies  # For compatibility with rest of notebook
dbscan_labels = hdbscan_labels

# Bonus: Show outlier score distribution
plt.figure(figsize=(10, 4))
plt.hist(outlier_scores, bins=50, edgecolor='black', alpha=0.7)
plt.xlabel('HDBSCAN Outlier Score')
plt.ylabel('Count')
plt.title('Distribution of Outlier Scores\\n(Higher = More likely to be anomaly)')
plt.axvline(x=np.percentile(outlier_scores, 90), color='red', linestyle='--', 
            label=f'90th percentile: {np.percentile(outlier_scores, 90):.2f}')
plt.legend()
plt.show()

## 5. Compute Consensus

In [ ]:
# Count votes from all methods
n_methods = 5 if elliptic_success else 4

anomaly_votes = (
    iso_anomalies.astype(int) +
    lof_anomalies.astype(int) +
    stat_anomalies.astype(int) +
    elliptic_anomalies.astype(int) +
    dbscan_anomalies.astype(int)
)

consensus_score = anomaly_votes / n_methods

# Assign discovered labels based on consensus
discovered_labels = np.where(
    consensus_score >= 0.8, 'high_confidence_anomaly',
    np.where(
        consensus_score >= 0.5, 'likely_anomaly',
        np.where(
            consensus_score >= 0.2, 'uncertain',
            'likely_normal'
        )
    )
)

print("Consensus Results:")
print("="*50)
for label in ['likely_normal', 'uncertain', 'likely_anomaly', 'high_confidence_anomaly']:
    count = (discovered_labels == label).sum()
    pct = count / len(discovered_labels) * 100
    emoji = "🟢" if 'normal' in label else "🔴" if 'anomaly' in label else "🟡"
    print(f"  {emoji} {label}: {count} ({pct:.1f}%)")

In [ ]:
# Visualize consensus distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram of consensus scores
axes[0].hist(consensus_score, bins=20, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Consensus Score (fraction of methods agreeing)')
axes[0].set_ylabel('Count')
axes[0].set_title('Distribution of Consensus Scores')
axes[0].axvline(x=0.5, color='red', linestyle='--', label='Anomaly threshold')
axes[0].legend()

# Pie chart of discovered labels
label_counts = pd.Series(discovered_labels).value_counts()
colors = {'likely_normal': '#2ecc71', 'uncertain': '#f1c40f', 
          'likely_anomaly': '#e74c3c', 'high_confidence_anomaly': '#c0392b'}
pie_colors = [colors[l] for l in label_counts.index]
axes[1].pie(label_counts.values, labels=label_counts.index, autopct='%1.1f%%', 
            colors=pie_colors, startangle=90)
axes[1].set_title('Discovered Label Distribution')

plt.tight_layout()
plt.show()

## 6. Method Agreement Analysis

In [ ]:
# Create agreement matrix
methods_df = pd.DataFrame({
    'Isolation Forest': iso_anomalies,
    'Local Outlier Factor': lof_anomalies,
    'Statistical': stat_anomalies,
    'Elliptic Envelope': elliptic_anomalies,
    'DBSCAN': dbscan_anomalies
})

# Correlation between methods
plt.figure(figsize=(10, 8))
correlation = methods_df.corr()
sns.heatmap(correlation, annot=True, cmap='RdYlGn_r', center=0,
            vmin=-1, vmax=1, square=True, linewidths=0.5)
plt.title('Agreement Between Anomaly Detection Methods')
plt.tight_layout()
plt.show()

print("\nPer-method anomaly rates:")
for col in methods_df.columns:
    rate = methods_df[col].mean() * 100
    print(f"  {col}: {rate:.1f}%")

## 7. K-Means Clustering Analysis

In [ ]:
# Find optimal number of clusters using silhouette score
print("Finding optimal number of clusters...")

max_clusters = min(10, len(X_scaled) - 1)
silhouette_scores = []

for k in range(2, max_clusters + 1):
    kmeans = KMeans(n_clusters=k, random_state=RANDOM_STATE, n_init=10)
    labels = kmeans.fit_predict(X_scaled)
    score = silhouette_score(X_scaled, labels)
    silhouette_scores.append((k, score))
    print(f"  k={k}: silhouette score = {score:.4f}")

optimal_k = max(silhouette_scores, key=lambda x: x[1])[0]
print(f"\nOptimal number of clusters: {optimal_k}")

# Plot silhouette scores
plt.figure(figsize=(10, 5))
ks, scores = zip(*silhouette_scores)
plt.plot(ks, scores, 'bo-', markersize=10)
plt.axvline(x=optimal_k, color='red', linestyle='--', label=f'Optimal k={optimal_k}')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Score vs Number of Clusters')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Perform final clustering with optimal k
kmeans_final = KMeans(n_clusters=optimal_k, random_state=RANDOM_STATE, n_init=10)
cluster_labels = kmeans_final.fit_predict(X_scaled)

print(f"Cluster distribution:")
for i in range(optimal_k):
    count = (cluster_labels == i).sum()
    pct = count / len(cluster_labels) * 100
    print(f"  Cluster {i}: {count} samples ({pct:.1f}%)")

In [ ]:
# Visualize clusters using PCA
pca_2d = PCA(n_components=2)
X_2d = pca_2d.fit_transform(X_scaled)

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: K-Means clusters
scatter1 = axes[0].scatter(X_2d[:, 0], X_2d[:, 1], c=cluster_labels, 
                           cmap='tab10', alpha=0.6, s=30)
axes[0].set_xlabel(f'PC1 ({pca_2d.explained_variance_ratio_[0]*100:.1f}%)')
axes[0].set_ylabel(f'PC2 ({pca_2d.explained_variance_ratio_[1]*100:.1f}%)')
axes[0].set_title('K-Means Clusters')
plt.colorbar(scatter1, ax=axes[0], label='Cluster')

# Plot 2: Discovered anomalies
colors = {'likely_normal': '#2ecc71', 'uncertain': '#f1c40f', 
          'likely_anomaly': '#e74c3c', 'high_confidence_anomaly': '#c0392b'}
for label in ['likely_normal', 'uncertain', 'likely_anomaly', 'high_confidence_anomaly']:
    mask = discovered_labels == label
    axes[1].scatter(X_2d[mask, 0], X_2d[mask, 1], c=colors[label], 
                    label=label, alpha=0.6, s=30)
axes[1].set_xlabel(f'PC1 ({pca_2d.explained_variance_ratio_[0]*100:.1f}%)')
axes[1].set_ylabel(f'PC2 ({pca_2d.explained_variance_ratio_[1]*100:.1f}%)')
axes[1].set_title('Discovered Anomalies')
axes[1].legend()

plt.tight_layout()
plt.show()

## 8. Feature Analysis: What Makes Anomalies Different?

In [ ]:
# Compare features between normal and anomalous samples
normal_mask = discovered_labels == 'likely_normal'
anomaly_mask = np.isin(discovered_labels, ['high_confidence_anomaly', 'likely_anomaly'])

feature_analysis = []

for i, col in enumerate(feature_cols):
    normal_vals = X_valid[normal_mask, i]
    anomaly_vals = X_valid[anomaly_mask, i]
    
    if len(normal_vals) > 0 and len(anomaly_vals) > 0:
        normal_mean = np.mean(normal_vals)
        anomaly_mean = np.mean(anomaly_vals)
        
        # Cohen's d effect size
        pooled_std = np.sqrt((np.std(normal_vals)**2 + np.std(anomaly_vals)**2) / 2)
        if pooled_std > 0:
            effect_size = abs(anomaly_mean - normal_mean) / pooled_std
        else:
            effect_size = 0
        
        feature_analysis.append({
            'feature': col,
            'normal_mean': normal_mean,
            'anomaly_mean': anomaly_mean,
            'difference': anomaly_mean - normal_mean,
            'effect_size': effect_size,
            'direction': 'higher' if anomaly_mean > normal_mean else 'lower'
        })

# Sort by effect size
feature_analysis = sorted(feature_analysis, key=lambda x: x['effect_size'], reverse=True)

print("Top 10 Most Discriminative Features:")
print("="*70)
print(f"{'Feature':<30} {'Effect Size':>12} {'Direction':>10} {'Normal':>10} {'Anomaly':>10}")
print("-"*70)
for item in feature_analysis[:10]:
    print(f"{item['feature']:<30} {item['effect_size']:>12.3f} {item['direction']:>10} "
          f"{item['normal_mean']:>10.2f} {item['anomaly_mean']:>10.2f}")

In [ ]:
# Visualize top features
top_features = [f['feature'] for f in feature_analysis[:6]]

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, feat in enumerate(top_features):
    feat_idx = feature_cols.index(feat)
    
    normal_vals = X_valid[normal_mask, feat_idx]
    anomaly_vals = X_valid[anomaly_mask, feat_idx]
    
    axes[i].hist(normal_vals, bins=30, alpha=0.5, label='Normal', color='green', density=True)
    axes[i].hist(anomaly_vals, bins=30, alpha=0.5, label='Anomaly', color='red', density=True)
    axes[i].set_xlabel(feat)
    axes[i].set_ylabel('Density')
    axes[i].set_title(f'{feat}\n(effect size: {feature_analysis[i]["effect_size"]:.2f})')
    axes[i].legend()

plt.tight_layout()
plt.show()

## 9. Temporal Analysis (if time column exists)

In [ ]:
if TIME_COL and TIME_COL in df.columns:
    # Create results dataframe for valid samples only
    results_df = df.loc[valid_mask].copy()
    results_df['discovered_label'] = discovered_labels
    results_df['consensus_score'] = consensus_score
    
    # Parse timestamps
    results_df[TIME_COL] = pd.to_datetime(results_df[TIME_COL], errors='coerce')
    results_df = results_df.dropna(subset=[TIME_COL])
    
    if len(results_df) > 0:
        # Group by day
        results_df['date'] = results_df[TIME_COL].dt.date
        
        daily_stats = results_df.groupby('date').agg({
            'consensus_score': ['count', 'mean'],
            'discovered_label': lambda x: x.isin(['high_confidence_anomaly', 'likely_anomaly']).sum()
        }).reset_index()
        daily_stats.columns = ['date', 'total', 'avg_score', 'anomalies']
        daily_stats['anomaly_rate'] = daily_stats['anomalies'] / daily_stats['total']
        
        # Plot temporal distribution
        fig, axes = plt.subplots(2, 1, figsize=(14, 8))
        
        # Plot 1: Daily anomaly rate
        axes[0].bar(range(len(daily_stats)), daily_stats['anomaly_rate'], color='coral')
        axes[0].set_ylabel('Anomaly Rate')
        axes[0].set_title('Daily Anomaly Rate')
        axes[0].axhline(y=0.3, color='red', linestyle='--', label='Suspicious threshold (30%)')
        axes[0].legend()
        
        # Plot 2: Daily sample count
        axes[1].bar(range(len(daily_stats)), daily_stats['total'], color='steelblue', label='Total')
        axes[1].bar(range(len(daily_stats)), daily_stats['anomalies'], color='coral', label='Anomalies')
        axes[1].set_xlabel('Day')
        axes[1].set_ylabel('Count')
        axes[1].set_title('Daily Sample Distribution')
        axes[1].legend()
        
        plt.tight_layout()
        plt.show()
        
        # Find suspicious periods
        suspicious = daily_stats[daily_stats['anomaly_rate'] > 0.3]
        if len(suspicious) > 0:
            print("\n⚠️ SUSPICIOUS PERIODS (>30% anomaly rate):")
            for _, row in suspicious.iterrows():
                print(f"   {row['date']}: {row['anomalies']}/{row['total']} anomalies ({row['anomaly_rate']*100:.1f}%)")
        else:
            print("\n✅ No suspicious time periods found.")
else:
    print("No time column available for temporal analysis.")

## 10. Compare with Existing Labels (if available)

In [ ]:
if EXISTING_LABEL_COL and EXISTING_LABEL_COL in df.columns:
    # Get existing labels for valid samples
    existing_labels = df.loc[valid_mask, EXISTING_LABEL_COL].values
    
    print("Comparison: Existing Labels vs Discovered Labels")
    print("="*70)
    
    comparison_df = pd.DataFrame({
        'existing': existing_labels,
        'discovered': discovered_labels
    })
    
    # Cross-tabulation
    cross_tab = pd.crosstab(comparison_df['existing'], comparison_df['discovered'], 
                            margins=True, normalize='index')
    
    print("\nNormalized by existing label (row-wise):")
    print(cross_tab.round(3))
    
    # Visualize
    plt.figure(figsize=(10, 6))
    cross_tab_counts = pd.crosstab(comparison_df['existing'], comparison_df['discovered'])
    sns.heatmap(cross_tab_counts, annot=True, fmt='d', cmap='YlOrRd')
    plt.title('Existing Labels vs Discovered Labels')
    plt.xlabel('Discovered Label')
    plt.ylabel('Existing Label')
    plt.tight_layout()
    plt.show()
    
    # Recommendations
    print("\n📋 RECOMMENDATIONS:")
    for existing_label in df[EXISTING_LABEL_COL].unique():
        mask = existing_labels == existing_label
        discovered_dist = pd.Series(discovered_labels[mask]).value_counts(normalize=True)
        
        anomaly_rate = discovered_dist.get('high_confidence_anomaly', 0) + discovered_dist.get('likely_anomaly', 0)
        normal_rate = discovered_dist.get('likely_normal', 0)
        
        if 'normal' in str(existing_label).lower() and anomaly_rate > 0.2:
            print(f"   ⚠️ '{existing_label}': {anomaly_rate*100:.1f}% look anomalous - review these samples!")
        elif 'attack' in str(existing_label).lower() or 'hijack' in str(existing_label).lower():
            if normal_rate > 0.3:
                print(f"   ⚠️ '{existing_label}': {normal_rate*100:.1f}% look normal - possible mislabeling!")
            else:
                print(f"   ✅ '{existing_label}': Looks correctly labeled ({anomaly_rate*100:.1f}% detected as anomalous)")
        else:
            print(f"   ℹ️ '{existing_label}': {normal_rate*100:.1f}% normal, {anomaly_rate*100:.1f}% anomalous")
else:
    print("No existing labels to compare with.")

## 11. Save Results

In [ ]:
# Create final results dataframe
results_df = df.copy()

# Add results for valid samples
results_df['iso_forest_score'] = np.nan
results_df['lof_score'] = np.nan
results_df['statistical_score'] = np.nan
results_df['elliptic_score'] = np.nan
results_df['cluster'] = -1
results_df['anomaly_votes'] = np.nan
results_df['consensus_score'] = np.nan
results_df['discovered_label'] = 'invalid'

results_df.loc[valid_mask, 'iso_forest_score'] = iso_scores
results_df.loc[valid_mask, 'lof_score'] = lof_scores
results_df.loc[valid_mask, 'statistical_score'] = stat_scores
results_df.loc[valid_mask, 'elliptic_score'] = elliptic_scores
results_df.loc[valid_mask, 'cluster'] = cluster_labels
results_df.loc[valid_mask, 'anomaly_votes'] = anomaly_votes
results_df.loc[valid_mask, 'consensus_score'] = consensus_score
results_df.loc[valid_mask, 'discovered_label'] = discovered_labels

# Save to file
input_path = Path(INPUT_FILE)
output_dir = Path(OUTPUT_DIR) if OUTPUT_DIR else input_path.parent
output_path = output_dir / f"{input_path.stem}_discovered.csv"

results_df.to_csv(output_path, index=False)
print(f"✅ Results saved to: {output_path}")

# Save anomalies separately
anomalies_df = results_df[results_df['discovered_label'].isin(['high_confidence_anomaly', 'likely_anomaly'])]
if len(anomalies_df) > 0:
    anomalies_path = output_dir / f"{input_path.stem}_anomalies.csv"
    anomalies_df.to_csv(anomalies_path, index=False)
    print(f"✅ Anomalies saved to: {anomalies_path} ({len(anomalies_df)} samples)")

## 12. Summary & Recommendations

In [ ]:
# Final summary
print("\n" + "="*70)
print("FINAL SUMMARY")
print("="*70)

likely_normal_count = (discovered_labels == 'likely_normal').sum()
anomaly_count = np.isin(discovered_labels, ['high_confidence_anomaly', 'likely_anomaly']).sum()
uncertain_count = (discovered_labels == 'uncertain').sum()

likely_normal_pct = likely_normal_count / len(discovered_labels) * 100
anomaly_pct = anomaly_count / len(discovered_labels) * 100

print(f"\n📊 Data analyzed: {len(df)} samples")
print(f"   Valid samples: {len(X_valid)}")
print(f"   Features used: {len(feature_cols)}")

print(f"\n🏷️  Discovered labels:")
print(f"   🟢 Likely Normal: {likely_normal_count} ({likely_normal_pct:.1f}%)")
print(f"   🟡 Uncertain: {uncertain_count}")
print(f"   🔴 Anomalies: {anomaly_count} ({anomaly_pct:.1f}%)")

print(f"\n📋 LABELING STRATEGY RECOMMENDATION:")

if likely_normal_pct > 80:
    print("""
    ✅ Your data appears to be PREDOMINANTLY NORMAL (>{:.0f}%).
    
    Recommended approach:
    1. Use 'likely_normal' samples as your NORMAL training data
    2. Investigate 'high_confidence_anomaly' samples manually
    3. Keep 'uncertain' samples for validation or discard them
    4. Cross-reference anomalies with known BGP incidents
    """.format(likely_normal_pct))
elif anomaly_pct > 30:
    print("""
    ⚠️ Your data has a HIGH ANOMALY RATE ({:.0f}%).
    
    This could indicate:
    - Data collected during a known BGP incident
    - Collector issues or misconfiguration
    - Legitimate attack traffic in your dataset
    
    Recommended approach:
    1. DO NOT use this data as "normal" baseline without review
    2. Check timestamps against known BGP incidents
    3. Verify RIPE collector status for the time period
    4. Consider using only 'likely_normal' samples for training
    """.format(anomaly_pct))
else:
    print("""
    ℹ️ Your data has a TYPICAL distribution.
    
    Recommended approach:
    1. Use 'likely_normal' samples as NORMAL class
    2. Use 'high_confidence_anomaly' as ATTACK class (after verification)
    3. Keep 'uncertain' samples for validation or manual review
    4. Document your labeling decisions for reproducibility
    """)

print("\n" + "="*70)
print("Analysis complete!")
print("="*70)